In [1]:
pip install speechrecognition pyttsx3 geopy folium googlemaps


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ------------- -------------------------- 41.0/125.4 kB ? eta -:--:--
   ------------- -------------------------- 41.0/125.4 kB ? eta -:--:--
   ------------------------- ------------- 81.9/125.4 kB 762.6 kB/s eta 0:00:01
   ------------------------- ------------- 81.9/125.4 kB 762.6 kB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 612.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/110.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/110.9 kB ? eta -:--:--
   ------------------------------------ --- 102.4/110.9 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 110.9/110.9 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--

In [3]:
import speech_recognition as sr
import pyttsx3
import googlemaps
import folium
from geopy.geocoders import Nominatim

# Initialize text-to-speech engine
engine = pyttsx3.init()

def speak(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def listen():
    """Capture voice input."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        speak("Where would you like to go?")
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        
        try:
            destination = recognizer.recognize_google(audio)
            print(f"You said: {destination}")
            return destination
        except sr.UnknownValueError:
            speak("Sorry, I didn't understand. Please try again.")
            return None

def get_location(destination):
    """Get latitude and longitude of destination."""
    geolocator = Nominatim(user_agent="voice_nav")
    location = geolocator.geocode(destination)
    if location:
        return location.latitude, location.longitude
    else:
        speak("Location not found.")
        return None

def get_route(destination):
    """Fetch route using Google Maps API."""
    gmaps = googlemaps.Client(key="YOUR_GOOGLE_MAPS_API_KEY")
    current_location = "Your Current Location"  # Replace with actual location
    directions = gmaps.directions(current_location, destination, mode="driving")
    
    if directions:
        route = directions[0]['legs'][0]['steps']
        speak(f"Navigating to {destination}.")
        for step in route:
            instruction = step['html_instructions']
            speak(instruction)
    else:
        speak("Unable to fetch route.")

def show_map(destination):
    """Display map with destination marker."""
    coords = get_location(destination)
    if coords:
        map_ = folium.Map(location=coords, zoom_start=15)
        folium.Marker(coords, popup=destination).add_to(map_)
        map_.save("navigation_map.html")
        speak("Map generated. Check navigation_map.html.")

# Main execution
destination = listen()
if destination:
    get_route(destination)
    show_map(destination)


Listening...
